## 1章 ニューラルネットワークの復習 まとめ
- ベクトルや行列の復習
- ニューラルネットワークの仕組み
   - 入力層，隠れ層，出力層を持つ
   - 計算グラフの基本パーツ
       - 加算ノード
       - 乗算ノード
       - Repeatノード
       - Sumノード
       - MatMulノード
   - 順伝播と逆伝播
- ニューラルネットワークの実装
    - レイヤ(NNの組み立てが容易になる)
        - Affineレイヤ(全結合層)
        - Sigmoidレイヤ(活性化関数,非線形な変換を可能にする)
        - SoftmaxWithLossレイヤ
        - forward()とbackward()を持つ
        - paramsとgradsを持つ
    - ミニバッチ処理で行列としてまとめて計算
    - 誤差逆伝播法で効率的にNNの損失に関する勾配を求めることができる．
- スパイラルデータセット
    - 二層NNで学習
- 高速化
    - ビット精度を32にする
    - GPUによる並列計算はnumpyと同じインタフェースを持つcupyで簡単に行える．

## 2章 自然言語と単語の分散表現 まとめ
- 単語の意味をコンピュータに理解させるためにはどうすればいいか？
    - シソーラス
        - 人の手で一つずつ単語の関連性を定義
        - 手間がかかり，表現力に限界がある
    - カウントベース
        - 単語の意味は周囲の単語によって形成される，という「分布仮説」に基づく手法
        - コーパスから自動的に単語の意味を（類似度を測定できる行列として）抽出
        - 単語をベクトルで表現

カウントベースの手法の手順
- コーパスから単語の共起行列を作成
- 共起行列を正の相互情報量を計算することでPPMI(Positive Pointwise Mutual Information)行列に変換
- SVDによる次元削減によって，計算量とロバスト性の問題を解消し，PPMIの単語ベクトルを密なベクトルへ変換

カウントベースの手法によって，意味的にも，文法的にも近い単語はベクトル空間上で近い位置になる．

テキストデータを扱いやすくするための下準備の関数を実装
- cos_similarity(): ベクトル同士のコサイン類似度
- most_similar(): PPMI行列から類似単語のランキングを表示

# 3章 word2vec まとめ
word2vecのCBOW(Continuous Bag of Words)モデルと呼ばれるNNについて詳しく説明し，実装を行なった．  
- 推論ベースの手法
    - コンテキスト，ターゲット間の関係を推論する問題を通して，その副産物として単語の分散表現（ベクトル表現）を得る
    - 単語が追加された時，1から計算をやり直さなくてはならないカウントベースの手法と比べ，それまでの重みを初期値として利用できる点で効率的．
    - word2vec
        - 基本的には2層のNN
        - MatMulレイヤとSoftmaxWithLossレイヤによる分類問題として構成されるモデル
        - 小さなコーパスでも学習はできる．
        - 現状では処理効率がよくない．  
        - CBOWモデル
            - コンテキストからターゲットを推論する問題
        - skip-gram モデル
            - ターゲットからコンテキストを推論する問題


# 4章 word2vecの高速化 まとめ
word2vecは語彙が増えるごとに計算量が増加するので，次の改良を行った
- MatMulレイヤ -> Embeddingレイヤ
    - one-hotベクトルを全部MatMulする必要はない．重みの対応する行を抜き出すだけ
- 出力層を 多値分類 -> 2値分類 + Negative sampling
    - 正解の単語に対する確率と，少量の不正解の単語に対する確率だけ計算すればいい
    - 不正解の単語をサンプリングするにはコーパス中の出現数の分布に0.75などの累乗を施した確率分布からランダムで選び出す

これらは，word2vecの計算の中で必要なところのみを取り扱うことで近似的に答えを導く方法である．  

word2vecによって得られた単語の分散表現(入力層の重み)は
- 似たコンテキストで使われる単語は近い値を持つようになる
- 類推問題をベクトルの加算と減算によって解ける性質を持つ
- 転移学習に利用でき，その他の様々な自然言語処理のタスクに利用できる．

# 5章 RNN まとめ
- 言語モデル
    - 単語の羅列の自然さなどを確率として解釈するモデル．
    - RNNレイヤを利用した条件付き言語モデルは，それまで登場した単語の情報を記憶することができる．
    - 言語モデルの評価は，パープレキシティという正解の確率の逆数を用いた指標を使う．
        - パープレキシティ： 次の単語の候補の数の相場
- RNNレイヤ
    - ループする経路を持ち，隠れ状態として，その時刻で出力した単語(の分散表現)を内部に記憶し，学習に利用するレイヤ
        - 今自分なんて言った？を覚えておく
    - RNNのループ経路を展開することで，複数のRNNレイヤが繋がったネットワークを形作る．
    - これにより，長大なコンテキストを時系列データとして学習できる．  
        - これらの単語の次はこの単語がくる，という情報を学習していく
    - 展開されたRNNの重み更新は誤差逆伝播法で行うことができ，これをBack Propergation Through Time (BPTT)と呼ぶ．
- Time RNNレイヤ
    - 長い時系列データを学習する場合は，適当な長さでデータのまとまりを作り，それらのブロック単位で誤差逆伝播法を行う．これをTruncated BPTTと呼ぶ．
    - Truncated BPTTでは順伝播の時，データの順番は保ったまま，開始点をずらしてデータを学習させる．  
- RNNLM
    - RNNによる言語モデル，すなわち文章の自然性学習器を実装
    - これでは非常に大きなコーパスを扱うことができない．

# 6章 ゲート付きRNN まとめ
単純なRNNの改良を行った．  
逆伝播の際，行列同士の積がひたすら時間方向に続き，勾配が指数的に大きく/小さくなっていくことで勾配爆発/消失が発生してしまう．  

- 勾配爆発
    - 勾配クリッピングで解決
        - 勾配の閾値を設け，その閾値を超えたら超えた分に応じて係数をかけ，大きくなりすぎた勾配を小さくする．
- 勾配消失
    - ゲート付きRNNで解決
        - ゲートは情報をどれだけ通すかを決める0.0~1.0の値の集合，ゲートでの計算結果はシグモイド関数で[0,1]に正規化される．
        - GRU: 付録C, Gated Recurrent Unit, LSTMよりパラメータが削減されていて高速
        - LSTM： 今回の主役，代表的なものは，以下のような仕組みを持つ
            - 記憶セル
                - 時刻tでの出力単語の他に，その単語を出力した時に使った記憶を，出力単語とは別に時間方向に伝播させるためのインタフェース
                - 記憶セルに関する計算は全て行列の積ではなく要素ごとの掛け算(アダマール積)で行われるため，勾配消失が起きにくい．
                - これにより，リアルタイムな短期記憶(short-term memory)を長い(Long)RNNブロックで覚えておく(コンテキストとして考慮できる)ことができるようになる．
            - forgetゲート: f
                - 入力された記憶セルの中から，各情報をどれだけ忘れるかを調整するゲート
                - 忘却はより良い前進を産む
            - inputゲート: i
                - 入力された記憶セルに，新しい情報をどれだけの重要度で覚えさせるかを調整し，記憶セルに追加するゲート
                - 次の記憶セルに渡す情報を決定する．
                - 新しい情報: g
                    - 前の時刻に出力された単語(の分散表現)と入力単語から算出される情報で，ゲートとは違い,tanhで[-1, 1]に正規化される．
            - outputゲート: o
                - 更新された記憶セルの情報に，前の時刻に出力された単語(の分散表現)と入力単語をどれだけ加味するかを調整するゲート
                - 出力単語(の分散表現)を決定する．
- 言語モデルの改良テクニック
    - LSTMレイヤの多層化
        - モデルがより複雑な問題に対応できるようになる．
        - 一方で，モデルが複雑になると過学習の恐れがある．
    - Dropout
        - 複雑さにペナルティを与える正則化の効果があり，過学習を防ぐ
        - 深さ方向： 普通に実装
        - 時間方向: 変分DropoutでDropoutする重みの位置を層ごとに固定
    - 重み共有
        - 一部の層で重みを共有することでパラメータを削減し，複雑さを軽減させて過学習を防ぐ
        - 入力側のEmbeddingレイヤの重みをAffineレイヤで利用

最新のモデルでは，重みそのものをランダムで無視するDropConnectや，8章のAttentionをベースとするcontinuous cache pointerなどが利用されており，その精度はパープレキシティは50程度にまで抑えられている．  
本章で実装した言語モデルでは，LSTMを用いたもので135程度,改良テクニックを使ったもので75程度のパープレキシティを出すことができる．

## 7章 RNNによる文章生成 まとめ
- 文章生成
    - RNNの入力に任意の単語を入れ，次の単語を予測していく．
    - 単語を出力される確率分布に従ってランダムにサンプルすることで，多様な文章が作成される
    - RNNの性能が良くなれば出力される文章もより自然になる．
    - 「the meaning of life is」のように複数の単語を最初に入力してやるのも面白い．
- Seq2seq(Encoder-Decoderモデル)
    - EncoderとDecoderという2つのRNNを使ってある時系列データを別の時系列データに変換する
    - EncoderのRNNの最後の時刻から出力される隠れ状態は入力データを圧縮したデータになる．
    - 入力データを反転させることで精度が上がることがある
    - 隠れ状態をDecoderのRNN以外の層にも渡してやることで精度が上がることがある
    - 足し算を文字のみから解かせる問題を上記の方法で学習させると，正答率が90%を超える
    - 応用分野が広く，実践的な技術
        - 機械翻訳
        - 自動要約
        - 質疑応答
        - メール自動返信
        - チャットボットによる問題解決
        - アルゴリズムの学習，自動プログラミング
        - イメージキャプション

# 8章 Attention まとめ
Attentionの仕組みを，seq2seq上で実装し，効果を確かめ，先端の応用について学んだ．  
- Attention
    - 翻訳元と翻訳先の単語があるように，時系列データ間には対応関係が存在することが多くある．  
    - Attentionは翻訳元の単語が与えられたとき，翻訳先の単語を選ぶ動作を学習することで，その対応関係を学ぶ．  
    - 対応関係を学ぶため，EncoderはLSTMの全ての時刻の出力データをDecoderに渡す．
    - ベクトルの内積などによる類似度を使った重み付き和ベクトルによって，対応する単語を選択
    - Attentionの処理は微分可能であるため，誤差逆伝播法で学習が可能
    - 日付変換問題では，従来のseq2seqよりも格段に早く学習した．
    - 算出される重み(確率)を可視化することで，入出力の対応関係を見ることができる．  
- Attentionに関するテクニック
    - 双方向RNN,双方向LSTM
        - 逆方向のTimeLSTMと順方向のTimeLSTMの出力を結合
    - Attentionレイヤを別のところに挟んでみる
        - TimeLSTMの各時刻の間など
    - seq2seqを深層化
        - モデルの複雑化による過学習を防ぐため，Skipコネクションなどを利用する
            - Skipコネクション：層の出力を層をまたいで先のノードに渡し，加算する
- Attentionの応用
    - Google Neural Machine Translation(GNMT)
        - NNによって人間に近い精度を持ったGoogleの翻訳システム
    - Transformer
        - Self-Attention: 1つの時系列データを自分の入力と時系列の最初に渡す層
        - LSTMではなくすべてをAttentionによって構成することで並列化が可能になり素早く学習できる
        - GNMTを凌駕
    - Neural Turing Machine(NTM)
        - 外部メモリによるNNの拡張の研究例
        - データからアルゴリズムを学習可能
        - メモリの読み書きにAttentionが用いられる

# 付録 まとめ
- sigmoid関数とtanh関数の逆伝搬の式
    - 微分を求めるには，計算グラフによる方法と解析的な方法がある
- WordNetを動かす
    - PythonからWordNetを動かすには，nltk(Natural Language Toolkit)パッケージをインストールして使う
    - 単語の意味ネットワークが構築されており，様々な単語間類似度の手法が提供されている
- GRU(Gated Recurrent Unit)
    - LSTMよりもパラメータ数が少なく，高速なRNN
    - LSTMとの精度差は，タスクやハイパーパラメータの調整によって様々．どちらが良いとは言い切れない．